# Cleaning methodology for pollutant data

This notebook contains the key steps taken to gather and clean air pollution data. As the data set is quite large and can only be downloaded in small chunks, we are currently investigation different ways of downloading the data in order to increase efficiency. As we continue to explore the data, more substantial filtering and cleaning will take place. 

In [1]:
import pandas as pd
import numpy as np

## Step 1: Identify boundary for monitoring. 

Having manually sifted through the data, we were able to split several monitoring sites into "inner" and "outer" ranges based on their geographic location. The data points closest to our area of focus (Heathrow Airport) have been categorised as "inner" locations and will be vital in our investigation into emissions. The "outer" locations will be used as a way of comparisson to gain a deeper understanding of the scale of the impact arising from air pollution. 

|CCG | Borough| Inner location - monitoring station| Outer location - monitoring site|
|---:|:-----|:-----------|:------------|
|Hillingdon |Hillingdon| Hillingdon South Ruislip, Hillingdon 2 Hillingdon Hospital, Hillingdon Oxford Avenue, Hillingdon Harmondsworth, Hillingdon Harmondsworth Osiris, Hillingdon Hayes, Heathrow LHR2, Heathrow Bath Road, Hillingdon Sipson, Heathrow Green Gates| |
|East Berkshire | Slough |Slough Town Centre Wellington Street,Slough Brands Hill London Road,Slough Windmill Bath Road,Slough Colnbrook,Slough Town Centre A4,Slough Lakeside 1 Osiris,Slough Colnbrook Osiris,Slough Chalvey,Slough Lakeside 2,Slough Lakeside 2 Osiris,Slough - Dennis Way LP11,Slough - Monksfield Way LP20,Slough - The Hawthorns LP2,Slough - Erica Close LP3,Slough - Hatton Avenue LP13,Slough - St Andrews Way LP12,Slough - The Hawthorns LP10,Slough - Francis Way LP13,Slough - The Hawthorns LP1,Slough - Monksfield Way LP19,Slough - Brighton Spur LP3,Slough - Bower Way LP1,Slough - Hatton Avenue LP3,Slough - Cinder Track LP37|
|Hounslow| Hounslow | Hounslow Cranford, Hounslow Chiswick, Hounslow Brentford, Hounslow Heston, Hounslow Hatton Cross, Hounslow Feltham, Hounslow Gunnersbury |
|Ealing | Ealing | Ealing Horn Lane
|Buckinghamshire|South Bucks|Iver Thorney Lane North, Iver North Park Road, Iver Primary School|
|Surrey Heartlands|Richmond|- |Elmbridge|
| - | Spelthorne | Spelthorne Shepperton Squire's Bridge Road, Spelthorne knowle Green, Spelthorne Sunbury Cross, Heathrow Oaks Road|
| - |Waverly and Woking | - | H&F Shepherd’s Bush, Godalming Ockford Road 2|
|South West London| Richmond | London Teddington Bushy Park. |
|Hammersmith & Fulham|London Borough of Hammersmith and Fulham|-| H&F Hammersmith Town Centre, H&F Shepherd’s Bush|
|Watford|Hertfordshire and Bedfordshire|-|Watford Town Hall|
|Oxfordshire|Oxfordshire|-|Oxford High St, Oxford St Ebbes (Cal Club), Oxford Center Roadside, Oxford St Ebbes|
| Berkshire West| Readiing| - | Reading Caversham Road, Reading Oxford Road, Reading London Road, Reading New Town|

## Step 2: Format the Data

In [2]:
boroughs = ['hammersmithAndFulham', 'oxford', 'reading', 'watford']

In [3]:
li = []

for borough in boroughs:
    df = pd.read_excel('./raw_data/{}.xlsx'.format(borough), header=[0,1], sheet_name=None)
    df = pd.concat(df.values(), axis=0)
    df = pd.DataFrame(df, columns=df.keys())
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

XLRDError: Unsupported format, or corrupt file: Expected BOF record; found b'Hourly m'

In [ ]:
# frame.to_excel("concatinated_pollution_data.xlsx")

In [ ]:
# df = pd.read_excel('concatinated_pollution_data.xlsx', header=[0,1], sheet_name=None)
# df = pd.concat(df.values(), axis=0)
# df = pd.DataFrame(df, columns=df.keys())
# df.head()

In [ ]:
# pd.set_option('display.max_columns', 100)

## Step 3: Unify missing data

In [ ]:
df = frame.replace('No data', np.nan)
df = df.replace('No Data', np.nan)
df.head()

## Step 4: Identify closed monitoring stations

In [ ]:
non_null_columns = [col for col in df.columns if df.loc[:, col].notna().any()]
open_monitoring_sites = df[non_null_columns]
open_monitoring_sites

## Step 5: Setting Date and Time columns to Datetime

In [ ]:
open_monitoring_sites[('Unnamed: 1_level_0','Time')] = [str(x)[-8:] for x in open_monitoring_sites[('Unnamed: 1_level_0','Time')]]

In [ ]:
open_monitoring_sites

In [ ]:
open_monitoring_sites[('Unnamed: 1_level_0','Time')] = pd.to_datetime(open_monitoring_sites[('Unnamed: 1_level_0','Time')], format='%H:%M:%S').dt.strftime("%H:%M:%S")

In [ ]:
open_monitoring_sites.columns

In [ ]:
open_monitoring_sites.set_index([('Unnamed: 0_level_0','Date'), ('Unnamed: 1_level_0','Time')], inplace=True)
open_monitoring_sites

In [ ]:
open_monitoring_sites = open_monitoring_sites.rename_axis(['Date','Time'])
open_monitoring_sites

## Step 6: Removing Multi-tier Columns

In [ ]:
open_monitoring_sites_stack = open_monitoring_sites.stack(0, dropna=True).rename_axis(('Date', 'Time','Location'))
open_monitoring_sites_stack

In [ ]:
open_monitoring_sites_stack.reset_index(level=['Location'])

## Renaming status columns

In [ ]:
list(open_monitoring_sites_stack.columns)

In [ ]:
pollutants = ['Carbon monoxide',
 'Nitric Oxide',
 'Nitrogen dioxide',
 'Non-volatile PM10',
 'Non-volatile PM2.5',
 'Oxides of Nitrogen',
 'Ozone',
 'PM1 Particulates',
 'PM10 Particulate matter',
 'PM10 particulate matter (Hourly measured)',
 'PM2.5 Particulate matter',
 'PM2.5 Particulate matter.1',
 'PM2.5 particulate matter (Hourly measured)',
 'Sulphur dioxide',
 'Volatile PM10']

In [ ]:
units = {'Status': 'Carbon monoxide units',
 'Status.1' : 'Nitric Oxide units',
 'Status.10' :'Nitrogen dioxide units',
 'Status.12' :'Non-volatile PM10 units',
 'Status.14' : 'Non-volatile PM2.5 units',
 'Status.2' : 'Oxides of Nitrogen units',
 'Status.3' : 'Ozone units',
 'Status.4' : 'PM1 Particulates units',
 'Status.5' : 'PM10 Particulate matter units',
 'Status.6' : 'PM10 particulate matter (Hourly measured) units',
 'Status.7' : 'PM2.5 Particulate matter units',
 'Status.8' : 'PM2.5 Particulate matter.1 units',
 'Status.9' : 'PM2.5 particulate matter (Hourly measured) units'
 }

In [ ]:
# open_monitoring_sites_stack.columns = open_monitoring_sites_stack.columns.map(units)

In [ ]:
open_monitoring_sites_stack.to_csv("stacked_inner_pollution_data.csv")

In [ ]:
# clean_data = pd.read_excel('concatinated_pollution_data.xlsx', header=[0,1], sheet_name=None)
# clean_dataclean_data = pd.concat(clean_data.values(), axis=0)
# clean_data = pd.DataFrame(clean_data, columns=clean_data.keys())
# clean_data.head()

## Next Steps

- Use API call to gather data from monitoring stations
- Investigate other data sources for mass downloads and ease
- Aggregate data on a daily basis
- Investigate similarities/differences between outer and inner locations, perform EDA on each
- Aggregate data annually and merge with Health and Flight data
